Before running this notebook, you should first run `./mnist_utils.py --option features --reduced 100 --binary` using the [mnist_utils.py from CrypTen](https://github.com/facebookresearch/CrypTen/blob/b1466440bde4db3e6e1fcb1740584d35a16eda9e/tutorials/mnist_utils.py) to prepare the dataset.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import crypten
import syft
from time import time
from syft import WebsocketClientWorker
from syft.frameworks.crypten.context import run_multiworkers


torch.manual_seed(0)
torch.set_num_threads(1)
hook = syft.TorchHook(torch)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/youben/anaconda3/envs/pysyft-dev-py37/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


In [2]:
# Define an example network
class ExampleNet(nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 2)  # For binary classification, final layer needs only 2 outputs

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [3]:
# Syft workers
print("[%] Connecting to workers ...")
ALICE = syft.VirtualWorker(hook=hook, id="alice")
BOB = syft.VirtualWorker(hook=hook, id="bob")
print("[+] Connected to workers")

# Add crypten support to the worker
ALICE.add_crypten_support()
BOB.add_crypten_support()

print("[%] Sending labels and training data ...")
# Prepare and send labels
label_eye = torch.eye(2)
labels = torch.load("/tmp/train_labels.pth")
labels = labels.long()
labels_one_hot = label_eye[labels]
labels_one_hot.tag("labels")
al_ptr = labels_one_hot.send(ALICE)
bl_ptr = labels_one_hot.send(BOB)

# Prepare and send training data
alice_train = torch.load("/tmp/alice_train.pth").tag("alice_train")
at_ptr = alice_train.send(ALICE)
bob_train = torch.load("/tmp/bob_train.pth").tag("bob_train")
bt_ptr = bob_train.send(BOB)

print("[+] Data ready")

# Initialize model
dummy_input = torch.empty(1, 1, 28, 28)
pytorch_model = ExampleNet()

[%] Connecting to workers ...
[+] Connected to workers
[%] Sending labels and training data ...
[+] Data ready


In [4]:
@run_multiworkers([ALICE, BOB], master_addr="127.0.0.1", model=pytorch_model, dummy_input=dummy_input)
def run_encrypted_training():
    rank = crypten.communicator.get().get_rank()
    
    worker_id = syft.local_worker.rank_to_worker_id[rank]
    worker = syft.local_worker.get_worker(worker_id)
    labels_one_hot = worker.search("labels")[0]

    # Load data:
    x_alice_enc = crypten.load("alice_train", 0)
    x_bob_enc = crypten.load("bob_train", 1)

    # Combine the feature sets: identical to Tutorial 3
    x_combined_enc = crypten.cat([x_alice_enc, x_bob_enc], dim=2)

    # Reshape to match the network architecture
    x_combined_enc = x_combined_enc.unsqueeze(1)

    # model is sent from the master worker
    model.encrypt()
    # Set train mode
    model.train()

    # Define a loss function
    loss = crypten.nn.MSELoss()

    # Define training parameters
    learning_rate = 0.001
    num_epochs = 2
    batch_size = 10
    num_batches = x_combined_enc.size(0) // batch_size

    for i in range(num_epochs):
        # Print once for readability
        if rank == 0:
            print(f"Epoch {i} in progress:")
            pass

        for batch in range(num_batches):
            # define the start and end of the training mini-batch
            start, end = batch * batch_size, (batch + 1) * batch_size

            # construct AutogradCrypTensors out of training examples / labels
            x_train = x_combined_enc[start:end]
            y_batch = labels_one_hot[start:end]
            y_train = crypten.cryptensor(y_batch, requires_grad=True)

            # perform forward pass:
            output = model(x_train)

            loss_value = loss(output, y_train)

            # set gradients to "zero"
            model.zero_grad()

            # perform backward pass:
            loss_value.backward()

            # update parameters
            model.update_parameters(learning_rate)

            # Print progress every batch:
            batch_loss = loss_value.get_plain_text()
            if rank == 0:
                print(f"\tBatch {(batch + 1)} of {num_batches} Loss {batch_loss.item():.4f}")

    model.decrypt()
    # printed contain all the printed strings during training
    return printed, model

In [5]:
print("[%] Starting computation")
func_ts = time()
result = run_encrypted_training()
func_te = time()
print(f"[+] run_encrypted_training() took {int(func_te - func_ts)}s")
printed = result[0][0]
model = result[0][1]
print(printed)

[%] Starting computation


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.
/home/youben/anaconda3/envs/pysyft-dev-py37/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
/home/youben/anaconda3/envs/pysyft-dev-py37/lib/python3.7/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead"

[+] run_encrypted_training() took 50s
Epoch 0 in progress:
	Batch 1 of 10 Loss 0.4638
	Batch 2 of 10 Loss 0.4665
	Batch 3 of 10 Loss 0.4065
	Batch 4 of 10 Loss 0.3489
	Batch 5 of 10 Loss 0.3312
	Batch 6 of 10 Loss 0.2796
	Batch 7 of 10 Loss 0.2767
	Batch 8 of 10 Loss 0.2430
	Batch 9 of 10 Loss 0.2457
	Batch 10 of 10 Loss 0.2002
Epoch 1 in progress:
	Batch 1 of 10 Loss 0.1624
	Batch 2 of 10 Loss 0.1516
	Batch 3 of 10 Loss 0.1549
	Batch 4 of 10 Loss 0.1922
	Batch 5 of 10 Loss 0.1319
	Batch 6 of 10 Loss 0.1634
	Batch 7 of 10 Loss 0.2243
	Batch 8 of 10 Loss 0.1454
	Batch 9 of 10 Loss 0.1717
	Batch 10 of 10 Loss 0.1334



In [6]:
cp = syft.VirtualWorker(hook=hook, id="cp")
model.fix_prec()
model.share(ALICE, BOB, crypto_provider=cp)
print(model)
print(list(model.parameters())[0])

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:70320103064 -> alice:76402485570]
	-> [PointerTensor | me:99473637600 -> bob:24884125725]
	*crypto provider: cp*


/home/youben/git-repo/PySyft/syft/frameworks/torch/tensors/interpreters/additive_shared.py:79: UserWarning: Use dtype instead of field
  warnings.warn("Use dtype instead of field")
/home/youben/git-repo/PySyft/syft/frameworks/torch/tensors/interpreters/additive_shared.py:91: UserWarning: Default args selected
  warnings.warn("Default args selected")
